# Böhringer - Accumulations - TODOS!
R&D hat zwei Werte. Bedeutung?

## Info
* Total stimmt bei Anzahl Empfänger manchmal nicht - falsch angeliefert

In [36]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [37]:
# Read pdf into DataFrame
df = tabula.read_pdf("Böhringer_efpia_report_switzerland_2017_de-1.pdf", pages='all', lattice=True)

In [50]:
df_export = df.copy()

#Shift all where column[1] not empty
column0 = df_export[df_export.columns[0]]
df_export[column0.isna()] = df_export[column0.isna()].shift(-1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[9:14],axis=1)
df_export = df_export.drop(df.columns[7],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#fill na
df_export = df_export.fillna("")

# Select
amount = df_export[df_export.type.str.contains('Gesamtbetrag entfallend auf')]
recipients = df_export[df_export.type.str.contains('Anzahl Empfänger in')]
rnd = df_export[df_export.type.str.contains('Geldwerte Leistungen')]

df_export = pd.concat([amount, recipients, rnd])

#Numberize and sum
df_export = cleanup_number(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)
#df_export = sum_amounts(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

df_export.loc[df_export.type == 'rnd', "total"] = df_export.loc[df_export.type == 'rnd', "sponsorship"]
df_export.loc[df_export.type == 'rnd', "donations_grants"] = np.NaN
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'boehringer')


Total nicht Summe der Werte
saved


In [89]:
t = df_export[df_export.type == "rnd"]
if t['registration_fees'].isna():
    print("x)")
#if pd.isna(t["registration_fees"]):
#    print("nan")

#if pd.isna(df_export.loc[df_export.type == 'rnd', 'total']):
#    print(4)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [67]:
df_export.head()

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,NaN,NaN,NaN,79006.65,138437.24,10447.12,227891.01,boehringer
1,hco_amount,1135965.56,654838.18,NaN,NaN,11761.53,673.50,1803238.77,boehringer
2,hcp_count,NaN,NaN,NaN,53.00,120.00,11.00,157.00,boehringer
3,hco_count,4.00,17.00,NaN,NaN,3.00,1.00,22.00,boehringer
4,rnd,NaN,NaN,NaN,NaN,NaN,NaN,6505476.67,boehringer
